In [1]:
from selenium import webdriver
import requests
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [2]:
path = '/home/gordo/development/Projects/web_scraping/Selenium_Projects/chromedriver'
web = 'https://projects.propublica.org/politwoops/user/realDonaldTrump'

In [3]:
options = Options()
options.headless = False
# options.add_argument('window_size=1920x1080')
driver = webdriver.Chrome(path, options= options)
driver.get(web)
driver.maximize_window()

In [4]:
# Pagin
pagins = driver.find_element_by_class_name('pagination')
pages = pagins.find_elements_by_xpath('.//a[@href]')
last_page = int(pages[-2].text)

In [ ]:
title = []
account = []
content = []
def scraper(title, account, content):
    container = WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.ID, "all-container")))
    # container = driver.find_element_by_class_name("adbl-impression-container")
    tweets = WebDriverWait(container, 25).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "tweet")))
    # tweets = container.find_elements_by_xpath('./li')
    title += [tweet.find_element_by_xpath('.//h4[@class="tweetTitle"]/a[@href]').text for tweet in tweets]
    account += [tweet.find_element_by_xpath('.//a[@class="accountHandle linkUnderline"]').text for tweet in tweets]
    content += [tweet.find_element_by_xpath('.//div[@class="tweet-content"]/p').text for tweet in tweets]
    

In [ ]:
current_page = 1

while current_page <= last_page:
    # time.sleep(3)
    
    CURRENT_PAGE = driver.current_url
    response = requests.get(CURRENT_PAGE)
    if response.status_code == 200:
        scraper(title, account, content)
        current_page += 1
    else:
        driver.get('https://projects.propublica.org/politwoops/user/realDonaldTrump?page=33')
        scraper(title,account,content)
        current_page += 1
    try:    
        next_page = driver.find_element_by_xpath('.//a[@class="next_page"]')
        next_page.click() 
    except:
        pass
driver.quit()

In [ ]:
df_books = pd.DataFrame({'Title': title, 'Account': account, 'Content': content})

df_books.to_csv('all_tweets_scraped.csv', index= False)